In [1]:
from csv_parser import csvRDD
import re

hsfn = 'DOE_High_School_Directory_2014-2015.csv'
satfn = 'SAT_Results.csv'

lines_hs = sc.textFile(hsfn, use_unicode=False).cache()
lines_sat = sc.textFile(satfn, use_unicode=False).cache()

rdd_hs = csvRDD(lines_hs)
rdd_sat = csvRDD(lines_sat)

def mapper1(records):
    for line in records:
        if line!='N/A':
            for i in line[10].split(', '):
                yield i, line[0]
            
def mapper2(records):
    for line in records:
        if line!='N/A':
            for i in line[11].split(' ; '):
                for j in i.split(', '):
                    yield j[0], line[0]
                
records_bus = rdd_hs.mapPartitions(mapper1)
records_sub = rdd_hs.mapPartitions(mapper2)

records_sat = rdd_sat.filter(lambda line: line[4]!='s') \
                     .map(lambda line: (line[0], line[4], line[2]))

records_bus.take(20)
# records_sub.take(20)

[('B41', '17K548'),
 ('B43', '17K548'),
 ('B44-SBS', '17K548'),
 ('B45', '17K548'),
 ('B48', '17K548'),
 ('B49', '17K548'),
 ('B69', '17K548'),
 ('Bx13', '09X543'),
 ('Bx15', '09X543'),
 ('Bx17', '09X543'),
 ('Bx21', '09X543'),
 ('Bx35', '09X543'),
 ('Bx4', '09X543'),
 ('Bx41', '09X543'),
 ('Bx4A', '09X543'),
 ('Bx6', '09X543'),
 ('Bx1', '09X327'),
 ('Bx11', '09X327'),
 ('Bx13', '09X327'),
 ('Bx18', '09X327')]

In [2]:
busdf = sqlContext.createDataFrame(records_bus, ("bus","dbn"))

subdf = sqlContext.createDataFrame(records_sub, ("subway","dbn"))

satdf = sqlContext.createDataFrame(records_sat, ("dbn","math_avg","total_takers"))

joindf1 = busdf.join(satdf, busdf.dbn == satdf.dbn, 'inner').drop(satdf.dbn)
joindf1.registerTempTable('bus_sat')

joindf2 = subdf.join(satdf, subdf.dbn == satdf.dbn, 'inner').drop(satdf.dbn)
joindf2.registerTempTable('sub_sat')

In [3]:
avg_by_bus = sqlContext.sql('select bus, (sum(math_avg*total_takers)/sum(total_takers)) as avg \
from bus_sat group by bus order by bus')

avg_by_bus.collect()

[Row(bus=u'B1', avg=461.4232804232804),
 Row(bus=u'B100', avg=475.0),
 Row(bus=u'B103', avg=531.7565379825654),
 Row(bus=u'B11', avg=503.80061983471074),
 Row(bus=u'B12', avg=390.6705276705277),
 Row(bus=u'B13', avg=397.04386951631045),
 Row(bus=u'B14', avg=390.61215629522434),
 Row(bus=u'B15', avg=382.9064245810056),
 Row(bus=u'B16', avg=448.6520979020979),
 Row(bus=u'B17', avg=385.2396694214876),
 Row(bus=u'B2', avg=475.0),
 Row(bus=u'B20', avg=372.43273905996756),
 Row(bus=u'B24', avg=432.02918403811793),
 Row(bus=u'B25', avg=541.0064543889845),
 Row(bus=u'B26', avg=389.8560533841754),
 Row(bus=u'B3', avg=382.46242774566474),
 Row(bus=u'B31', avg=475.0),
 Row(bus=u'B32', avg=409.11226252158895),
 Row(bus=u'B35', avg=390.5),
 Row(bus=u'B36', avg=432.6330128205128),
 Row(bus=u'B38', avg=520.8400561206594),
 Row(bus=u'B39', avg=438.80888429752065),
 Row(bus=u'B4', avg=466.2381268681501),
 Row(bus=u'B41', avg=520.8459335624284),
 Row(bus=u'B42', avg=384.48026315789474),
 Row(bus=u'B43',

In [4]:
avg_by_sub = sqlContext.sql('select subway, (sum(math_avg*total_takers)/sum(total_takers)) as avg \
from sub_sat group by subway')

avg_by_sub.collect()

[Row(subway=u'1', avg=499.84488281908614),
 Row(subway=u'2', avg=488.0718242975861),
 Row(subway=u'3', avg=513.4009556313994),
 Row(subway=u'4', avg=495.29238227146817),
 Row(subway=u'5', avg=461.0280319703463),
 Row(subway=u'6', avg=432.80367816091956),
 Row(subway=u'7', avg=457.35861778339654),
 Row(subway=u'A', avg=510.0150229357798),
 Row(subway=u'B', avg=491.95760524225574),
 Row(subway=u'C', avg=510.239433265865),
 Row(subway=u'D', avg=502.2631520035818),
 Row(subway=u'E', avg=501.2646720368239),
 Row(subway=u'F', avg=445.7865661411926),
 Row(subway=u'G', avg=503.4458706708646),
 Row(subway=u'J', avg=439.1299656694458),
 Row(subway=u'L', avg=426.3222871994802),
 Row(subway=u'M', avg=454.06567963458815),
 Row(subway=u'N', avg=483.49450806263144),
 Row(subway=u'Q', avg=482.14557840292673),
 Row(subway=u'R', avg=508.6067355282978),
 Row(subway=u'S', avg=466.4121255864309),
 Row(subway=u'Z', avg=438.12698819907644)]